In [1]:
import dgl
import math
import torch
import numpy as np
from os import path
from pathlib import Path
from dgl.data import DGLDataset
from ToyDGLDataset import ToyDGLDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

Using backend: pytorch


In [2]:
datasetName = 'ToyDataset01'
datasetDir = path.join('/home/andrew/GNN_Sandbox/GraphDatasets', datasetName)
dataset = ToyDGLDataset(datasetName, datasetDir)

Done loading data from cached files.


In [3]:
dataset.printProperties()

Num Graph classes: 2
Graph classes: [0, 1]
Number of graphs: 4000
Number of all nodes in all graphs: 187464
Number of all edges in all graphs: 10140816
Dim node features: 5
Node feature keys: ['P_t', 'Eta', 'Phi', 'Mass', 'Type']
Dim edge features: 3
Edge feature keys: ['DeltaEta', 'DeltaPhi', 'RapiditySquared']


In [4]:
print(len(dataset))
graph, label = dataset[0]
print(graph)
print(f'Label: {label}')
print(GetNodeFeatureVectors(graph))

4000
Graph(num_nodes=14, num_edges=182,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)})
Label: 0
tensor([[ 8.2500e+01,  2.4013e+00,  4.4686e+00,  6.9914e-01,  1.0000e+00],
        [ 3.3428e+01,  6.1951e+00,  4.7875e+00,  2.0964e-01,  0.0000e+00],
        [ 8.8582e+01, -4.9841e+00,  5.8098e+00,  4.0086e-01,  2.0000e+00],
        [ 9.1063e+01, -1.9321e+00,  1.6134e+00,  4.3644e-01,  0.0000e+00],
        [ 8.3695e+01,  8.4926e+00,  5.9178e+00,  9.9072e-01,  1.0000e+00],
        [ 8.3720e+01, -5.7298e+00,  3.1423e+00,  5.3718e-01,  2.0000e+00],
        [ 4.0792e+01, -6.3605e+00,  4.7412e+00,  7.0650e-01,  1.0000e+00],
        [ 1.6013e+01, -9.9891e+00,  4.0879e+00,  9.9766e-01,  1.0000e+00],
        [ 8.3934e+01,  5.7967e+00,  5.4796e+00,  6.5208e-01,  1.0000e+00],
        [ 1.1577e+01,  7.7988e+00,  1.1120e+00,  4.8540e-01,  0.0000e+00],
        [ 7.7765e+01,  2.9263e+00,  4.8746e+00,  7.1705e-01,

In [5]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
splitIndices = dataset.get_split_indices()

train_sampler = SubsetRandomSampler(splitIndices['train'])
test_sampler = SubsetRandomSampler(splitIndices['test'])

train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=32, drop_last=False)
test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=32, drop_last=False)

In [6]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

[Graph(num_nodes=1346, num_edges=67446,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)}), tensor([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 0])]


In [7]:
batched_graph, labels = batch
print('Number of nodes for each graph element in the batch:', batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:', batched_graph.batch_num_edges())
print(labels)
# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)

Number of nodes for each graph element in the batch: tensor([61, 58, 39, 60, 44, 62, 51, 32, 30, 23, 61, 38, 30, 17, 28, 28, 22, 34,
        18, 67, 29, 33, 76, 25, 72, 17, 70, 64, 76, 26, 45, 10])
Number of edges for each graph element in the batch: tensor([3660, 3306, 1482, 3540, 1892, 3782, 2550,  992,  870,  506, 3660, 1406,
         870,  272,  756,  756,  462, 1122,  306, 4422,  812, 1056, 5700,  600,
        5112,  272, 4830, 4032, 5700,  650, 1980,   90])
tensor([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 0])
The original graphs in the minibatch:
[Graph(num_nodes=61, num_edges=3660,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)}), Graph(num_nodes=58, num_edges=3306,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)}), Graph(num_nodes=39, num_

In [8]:
from dgl.nn import GraphConv
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [9]:
# Create the model with given dimensions
model = GCN(dataset.dim_nfeats, 16, dataset.num_graph_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(50):
    for batched_graph, labels in train_dataloader:
        nodeFeatVec = GetNodeFeatureVectors(batched_graph)
        pred = model(batched_graph, nodeFeatVec)
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #if epoch % 5 == 0:
    print(f'Epoch: {epoch}, Loss: {loss:.3f}')

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    nodeFeatVec = GetNodeFeatureVectors(batched_graph)
    pred = model(batched_graph, nodeFeatVec)
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print('Test accuracy:', num_correct / num_tests)

Epoch: 0, Loss: 0.600
Epoch: 1, Loss: 0.520
Epoch: 2, Loss: 0.454
Epoch: 3, Loss: 0.434
Epoch: 4, Loss: 0.297
Epoch: 5, Loss: 0.319
Epoch: 6, Loss: 0.314
Epoch: 7, Loss: 0.273
Epoch: 8, Loss: 0.164
Epoch: 9, Loss: 0.397
Epoch: 10, Loss: 0.146
Epoch: 11, Loss: 0.252
Epoch: 12, Loss: 0.140
Epoch: 13, Loss: 0.220
Epoch: 14, Loss: 0.051
Epoch: 15, Loss: 0.116
Epoch: 16, Loss: 0.107
Epoch: 17, Loss: 0.128
Epoch: 18, Loss: 0.173
Epoch: 19, Loss: 0.146
Epoch: 20, Loss: 0.151
Epoch: 21, Loss: 0.207
Epoch: 22, Loss: 0.172
Epoch: 23, Loss: 0.180
Epoch: 24, Loss: 0.039
Epoch: 25, Loss: 0.036
Epoch: 26, Loss: 0.075
Epoch: 27, Loss: 0.100
Epoch: 28, Loss: 0.105
Epoch: 29, Loss: 0.038
Epoch: 30, Loss: 0.207
Epoch: 31, Loss: 0.069
Epoch: 32, Loss: 0.073
Epoch: 33, Loss: 0.047
Epoch: 34, Loss: 0.129
Epoch: 35, Loss: 0.127
Epoch: 36, Loss: 0.040
Epoch: 37, Loss: 0.089
Epoch: 38, Loss: 0.112
Epoch: 39, Loss: 0.100
Epoch: 40, Loss: 0.046
Epoch: 41, Loss: 0.082
Epoch: 42, Loss: 0.355
Epoch: 43, Loss: 0.01

In [10]:
print(graph.nodes[[0,1]])
print(graph.nodes[0])
print(graph.nodes[1])
print(graph.nodes[10])
print(graph.nodes[3])

NodeSpace(data={'feat': tensor([[82.4999,  2.4013,  4.4686,  0.6991,  1.0000],
        [33.4276,  6.1951,  4.7875,  0.2096,  0.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[82.4999,  2.4013,  4.4686,  0.6991,  1.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[33.4276,  6.1951,  4.7875,  0.2096,  0.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[77.7649,  2.9263,  4.8746,  0.7170,  1.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[91.0627, -1.9321,  1.6134,  0.4364,  0.0000]], dtype=torch.float64)})


In [11]:
#print(graph.out_edges(7))
#print(graph.edges[(0,1)]) # src, dst
#print(graph.edges[0,1]) # src, dst
print(graph.edges[[0, 0], [1,2]]) # src, dst
print(graph.edges[[10], [3]])
print(graph.edges[[3], [10]])
print(graph.edges[0])
print(graph.edges[1])
#print(graph.edges[0]) # eid

EdgeSpace(data={'feat': tensor([[ 3.7938,  0.3189, 14.4946],
        [ 7.3855,  1.3411, 56.3441]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[ 4.8584,  3.2611, 34.2393]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[ 4.8584,  3.2611, 34.2393]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[ 3.7938,  0.3189, 14.4946]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[ 7.3855,  1.3411, 56.3441]], dtype=torch.float64)})


In [12]:
edgeList = GetNeighborNodes(graph, 0)
nodeFeat = graph.ndata['feat']
print(graph.nodes[0])
print(nodeFeat)
print(edgeList)


NodeSpace(data={'feat': tensor([[82.4999,  2.4013,  4.4686,  0.6991,  1.0000]], dtype=torch.float64)})
tensor([[ 8.2500e+01,  2.4013e+00,  4.4686e+00,  6.9914e-01,  1.0000e+00],
        [ 3.3428e+01,  6.1951e+00,  4.7875e+00,  2.0964e-01,  0.0000e+00],
        [ 8.8582e+01, -4.9841e+00,  5.8098e+00,  4.0086e-01,  2.0000e+00],
        [ 9.1063e+01, -1.9321e+00,  1.6134e+00,  4.3644e-01,  0.0000e+00],
        [ 8.3695e+01,  8.4926e+00,  5.9178e+00,  9.9072e-01,  1.0000e+00],
        [ 8.3720e+01, -5.7298e+00,  3.1423e+00,  5.3718e-01,  2.0000e+00],
        [ 4.0792e+01, -6.3605e+00,  4.7412e+00,  7.0650e-01,  1.0000e+00],
        [ 1.6013e+01, -9.9891e+00,  4.0879e+00,  9.9766e-01,  1.0000e+00],
        [ 8.3934e+01,  5.7967e+00,  5.4796e+00,  6.5208e-01,  1.0000e+00],
        [ 1.1577e+01,  7.7988e+00,  1.1120e+00,  4.8540e-01,  0.0000e+00],
        [ 7.7765e+01,  2.9263e+00,  4.8746e+00,  7.1705e-01,  1.0000e+00],
        [ 7.7500e+01,  4.9447e-01,  3.2061e+00,  4.0157e-01,  0.0000e+00

In [15]:
def ConcatNodeAndEdgeFeatures(graph, nodeFeat, nodeLabel: int):
    neighbors = GetNeighborNodes(graph, nodeLabel)
    nfeat = nodeFeat[nodeLabel]
    efeat = torch.reshape(graph.edges[neighbors].data['feat'], (-1,))
    #print(nfeat)
    #print(efeat)
    return torch.cat((nfeat, efeat))
    
x = ConcatNodeAndEdgeFeatures(graph, nodeFeat, 0)
print(x)

edgeFeatureVec = GetEdgeFeatureVectors(graph)

x = (1,2,3)
print(x)


tensor([ 82.4999,   2.4013,   4.4686,   0.6991,   1.0000,   3.7938,   0.3189,
         14.4946,   7.3855,   1.3411,  56.3441,   4.3334,   2.8552,  26.9309,
          6.0913,   1.4491,  39.2035,   8.1311,   1.3263,  67.8747,   8.7619,
          0.2726,  76.8443,  12.3904,   0.3807, 153.6669,   3.3954,   1.0109,
         12.5504,   5.3975,   3.3566,  40.3997,   0.5250,   0.4059,   0.4404,
          1.9069,   1.2625,   5.2301,  10.9706,   2.7195, 127.7507,   5.1435,
          0.1850,  26.4903], dtype=torch.float64)
(1, 2, 3)


In [14]:
print(graph.edata['DeltaPhi'])
print(graph.dstdata['P_t'])

KeyError: 'DeltaPhi'

In [ ]:
a = torch.tensor(np.arange(5,10))
b = torch.tensor(np.arange(9).reshape((3,3)))

print(a)
print(b)
b = torch.reshape(b, (-1,))
print(b)
print(torch.cat((a,b), dim=0))